## Junge Freiheit
#### Code for scraping Junge Freiheit
- https://jungefreiheit.de

In [49]:
# Import necessary libraries
import requests
import urllib
import pandas as pd
from bs4 import BeautifulSoup

In [50]:
# Using two keywords ("migraten" and "flüchtling") this time because this is a smaller newspaper with less articles
search_f = '/?s=fl%C3%BCchtling'
base_url = 'https://jungefreiheit.de/page/'
n = 2 # I started at 2 because the structure of the url is different for page 1 and there is no reason I would need
# it searching for 2015

#Compile urls 
urls_f = []
while n <= 39:
    url_f = base_url+str(n)+search_f
    n += 1
    urls_f.append(url_f)
    
print(len(urls_f))

# Search using keyword: Migranten
search_m = '/?s=migranten'
urls_m = []
m = 2
while m <= 149:
    url_m = base_url+str(m)+search_m
    m += 1
    urls_m.append(url_m)
    
print(len(urls_m))

38
148


In [51]:
#Create one list of urls
urls = urls_m +urls_f
print(len(urls))

186


In [52]:
# Loop through the urls of the pages of articles and if the date is 2015 get the links for the relevent articles

art_links = []
date = []
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    d = soup.find("span", {"class":"elementor-post-date"})
    if d.text[-6:-2]=='2015':
        date.append(d.text)
        divs = soup.find_all("div", {"class":"elementor-post__text"})
        for div in divs:
            a = div.find('a')
            art_links.append(a.get('href'))
            
        
#art_links                          

In [53]:
# a little sparse, but it is a smaller weekly newspaper
len(art_links)

63

In [54]:
# Loop through each link and get the date and text and append to a dictionary
date_dict = {}
art_dict_jf = {}
x = ' '
for link in art_links:
    page = urllib.request.urlopen(link)
    soup = BeautifulSoup(page)
    t = soup.find("div", {"class":"elementor-widget-theme-post-content"})
    #print(t)
    paras = t.findAll('p')
    a = [p.text for p in paras]
    d = soup.find("span", {"class":"elementor-icon-list-text elementor-post-info__item elementor-post-info__item--type-date"})
    text = x.join(a)
    date_dict[link] = d.text
    art_dict_jf[link] = text
    #print(text)
    
#art_dict_jf
#date_dict

In [55]:
# I guess there were 3 duplicates
len(art_dict_jf)

60

In [56]:
df_jf = pd.DataFrame.from_dict(art_dict_jf, orient='index')
df_jf.reset_index(inplace=True)
df_jf.columns = ['href', 'text']
word_c = df_jf.text.str.split().map(len)
df_jf['word_count'] = word_c

df_jf['date']= df_jf['href'].map(date_dict)

df_jf.head()

,href,text,word_count,date
0,https://jungefreiheit.de/politik/deutschland/2...,POTSDAM. Brandenburgs AfD-Chef Alexander Gaula...,385,18. November 2015
1,https://jungefreiheit.de/debatte/kommentar/201...,Die Norwegerin Linda Hagen ist immer noch ganz...,171,05. November 2015
2,https://jungefreiheit.de/politik/deutschland/2...,"ERFURT. Asylbewerber, die mit der Deutschen Ba...",191,04. November 2015
3,https://jungefreiheit.de/politik/ausland/2015/...,TRIPOLIS. Der libysche „Allgemeine Volkskongre...,262,04. November 2015
4,https://jungefreiheit.de/politik/deutschland/2...,Dreizehn Regierungschefs beschließen auf einem...,729,01. November 2015


In [57]:
df_jf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   href        60 non-null     object
 1   text        60 non-null     object
 2   word_count  60 non-null     int64 
 3   date        60 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.0+ KB


In [58]:
# Pickling
pd.to_pickle(df_jf, "jf_df.pkl")